#Transform Refunds Data
1. Extract specific portion of the string from refund_reason using split function
2. Extract specific portion of the string from refund_reason using regexp_extract function
3. Extract date and time from the refund_timestamp
4. Write transformed data to the Silver schema in hive metastore

In [0]:
%sql
FROM hive_metastore.bronze.refunds
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       refund_reason;

In [0]:
%sql
FROM hive_metastore.bronze.refunds
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       SPLIT(refund_reason, ':')[0] AS refund_reason,
       SPLIT(refund_reason, ':')[1] AS refund_source;

In [0]:
%sql
FROM hive_metastore.bronze.refunds
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       regexp_extract(refund_reason, '^([^:]+):', 1) AS refund_reason,
       regexp_extract(refund_reason, '^[^:]+:(.*)$', 1) AS refund_source;

In [0]:
%sql
FROM hive_metastore.bronze.refunds
SELECT refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
       refund_amount,
       regexp_extract(refund_reason, '^([^:]+):', 1) AS refund_reason,
       regexp_extract(refund_reason, '^[^:]+:(.*)$', 1) AS refund_source;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS hive_metastore.silver;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS hive_metastore.silver.refunds
AS
FROM hive_metastore.bronze.refunds
SELECT refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
       refund_amount,
       regexp_extract(refund_reason, '^([^:]+):', 1) AS refund_reason,
       regexp_extract(refund_reason, '^[^:]+:(.*)$', 1) AS refund_source;

In [0]:
%sql
SELECT * FROM hive_metastore.silver.refunds;

In [0]:
%sql
DESC EXTENDED hive_metastore.silver.refunds;